# Init

In [ ]:
import kutils
from kutils import applications as apps
from kutils import model_helper as mh
from kutils import image_utils
from kutils import generic
import pandas as pd, numpy as np

In [ ]:
root_path = '/mnt/home/research/ava_mlsp/'
ids = pd.read_csv(root_path + '/metadata/AVA_data_official_test.csv')

input_shape = (None, None, 3)
features_root = root_path + 'features/'
images_path = root_path + 'images/'

# Choose and load pretrained model

### InceptionV3 MLSP narrow

In [ ]:
model = apps.model_inception_multigap(input_shape)
pre = apps.process_input[apps.InceptionV3]
model_name = 'iv3_mlsp_narrow'

### InceptionV3 MLSP wide

In [ ]:
model = apps.model_inception_pooled(input_shape)
pre   = apps.process_input[apps.InceptionV3]
model_name = 'iv3_mlsp_wide'

### InceptionResNetV2 MLSP narrow

In [ ]:
model = apps.model_inceptionresnet_multigap(input_shape)
pre = apps.process_input[apps.InceptionResNetV2]
model_name = 'irnv2_mlsp_narrow'

### InceptionResNetV2 MLSP wide

In [ ]:
model = apps.model_inceptionresnet_pooled(input_shape)
pre   = apps.process_input[apps.InceptionResNetV2]
model_name = 'irnv2_mlsp_wide'

# Save features

### original sized images, no augmentation

In [ ]:
gen_params = dict(batch_size  = 1,
                  data_path   = images_path,
                  input_shape = ('orig',),
                  inputs = ('image_name',),
                  process_fn  = pre,
                  fixed_batches = False)

helper = mh.ModelHelper(model, model_name + '_orig', ids,
                    features_root = features_root,
                    gen_params    = gen_params)        

print 'Saving features'
batch_size = 1024
numel = len(ids)
for i in range(0,numel,batch_size):
    istop = min(i+batch_size, numel)
    print 'Processing images',i,':',istop
    ids_batch = ids[i:istop].reset_index(drop=True)
    helper.save_activations(ids=ids_batch, verbose=True,\
                            save_as_type=np.float16)

### originals, augmented with flip and crop

In [ ]:
crops = [(i,j) for i in (1,0) for j in (1,0)]
flips = (0,1)

for i, crop_pos in enumerate(crops):
    for doflip in flips:
        print 'crop', crop_pos, 'flip' if doflip else ''
        process_fn = lambda im: pre(image_utils.ImageAugmenter(im, verbose=False, remap=False).\
                                    crop(0.875, crop_pos).fliplr(doflip).result)
        gen_params = dict(batch_size  = 1,
                          data_path   = images_path,
                          input_shape = ('orig',),
                          inputs = ('image_name',), 
                          process_fn  = process_fn,
                          fixed_batches = False)

        helper = mh.ModelHelper(model, model_name + '_fc8_orig', ids,
                            features_root = features_root,
                            gen_params    = gen_params)

        print 'Saving features'
        batch_size = 1024
        numel = len(ids)
        for i in range(0,numel,batch_size):
            istop = min(i+batch_size, numel)
            print 'Processing images',i,':',istop
            ids_batch = ids[i:istop].reset_index(drop=True)

            helper.save_activations(ids=ids_batch, verbose=False,\
                                    groups=['%g_%g_%d' %(crop_pos+(doflip,))], 
                                    save_as_type=np.float16, over_write=False)

In [ ]:
# check contents of saved file
with generic.H5Helper(features_root + 'irnv2_mlsp_narrow_orig/i1[orig]_lfinal_o1[16928]_r1.h5','r') as h:
    print h.summary()